<a href="https://colab.research.google.com/github/miguellucero123/Analisis_est/blob/main/Trabajo_TDA304_G5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Montar Google Drive y preparar entorno
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

# Ruta del archivo (ajusta si es necesario)
ruta = '/content/drive/MyDrive/26_05_2025/datos_supermercadousa.csv'

# Cargar datos
df = pd.read_csv(ruta, encoding='latin-1', sep=';')

# Función de limpieza de números (para formato español)
def limpiar_numero(valor):
    if pd.isnull(valor):
        return np.nan
    s = str(valor).replace(',', '').strip()
    if s.count('.') > 1:
        partes = s.split('.')
        s = ''.join(partes[:-1]) + '.' + partes[-1]
    try:
        return float(s)
    except Exception:
        return np.nan

# Corregir nombres de columnas a español estándar
df = df.rename(columns={
    'Invoice ID': 'ID de factura',
    'Branch': 'Sucursal',
    'City': 'Ciudad',
    'Customer type': 'Tipo de comprador',
    'Gender': 'Género',
    'Product line': 'Línea de producto',
    'Unit price': 'Precio por unidad',
    'Canrtidad': 'Cantidad',
    'Tax 5%': 'Impuesto 5%',
    'Total': 'Total',
    'Date': 'Fecha',
    'Time': 'Hora',
    'Payment': 'Tipo de pago',
    'COGS': 'Costo de bienes vendidos',
    'Gross margin percentage': 'Porcentaje margen bruto',
    'Gross income': 'Ingreso bruto',
    'Rating': 'Calificación'
})

# Limpiar columnas numéricas
for col in ['Precio por unidad', 'Total', 'Impuesto 5%', 'Ingreso bruto', 'Calificación', 'Cantidad']:
    if col in df.columns:
        df[col] = df[col].apply(limpiar_numero)

# Eliminar duplicados y valores faltantes
df = df.dropna().drop_duplicates()

# Calcular columnas requeridas
df['Subtotal'] = df['Precio por unidad'] * df['Cantidad']
df['Impuesto 5%'] = df['Subtotal'] * 0.05
df['Total'] = df['Subtotal'] + df['Impuesto 5%']
df['Ingreso bruto'] = df['Impuesto 5%']

# Filtro de atípicos (outliers) en Total y Calificación
def filtro_iqr(datos, columna):
    Q1 = datos[columna].quantile(0.25)
    Q3 = datos[columna].quantile(0.75)
    IQR = Q3 - Q1
    inferior = Q1 - 1.5 * IQR
    superior = Q3 + 1.5 * IQR
    return datos[(datos[columna] >= inferior) & (datos[columna] <= superior)]

df = filtro_iqr(df, 'Total')
df = filtro_iqr(df, 'Calificación')

# Conversión de fecha
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
df = df.dropna(subset=['Fecha'])

In [ ]:
df.shape

In [ ]:
print("1.- ¿Qué representa cada registro del dataframe?")
print("Respuesta:")
print("Cada registro representa una transacción individual realizada en una de las sucursales del supermercado internacional. "
      "Incluye información sobre la compra, el cliente, el producto, el monto y la calificación otorgada por el cliente.")
print("\nNombres de las columnas en español:", df.columns.tolist())
display(df.head())

In [ ]:
print("2. ¿Cuál es el promedio de ventas diarias por la línea de producto?")
ventas_diarias = df.groupby(['Fecha', 'Linea de producto'])['Total'].sum().reset_index()
promedio_ventas_diarias = ventas_diarias.groupby('Linea de producto')['Total'].mean().sort_values(ascending=False)
print("Respuesta:")
print("El promedio de ventas diarias por línea de producto se obtuvo agrupando los datos por fecha y línea de producto, "
      "sumando las ventas diarias y calculando el promedio para cada línea.")
display(promedio_ventas_diarias)

In [ ]:
print("3. ¿Qué línea de producto generó más ingresos totales?")
ingresos_por_linea = df.groupby('Linea de producto')['Total'].sum().sort_values(ascending=False)
linea_mayor_ingreso = ingresos_por_linea.idxmax()
print("Respuesta:")
print(f"La línea de producto que generó más ingresos totales es: {linea_mayor_ingreso}")
print("Esto se obtuvo sumando los ingresos totales de cada línea de producto y seleccionando la de mayor valor.")
display(ingresos_por_linea)

In [ ]:
print("4. ¿Cuál fue la calificación promedio por la línea de producto y género?")
calif_promedio = df.groupby(['Linea de producto', 'Género'])['Calificación'].mean().unstack()
print("Respuesta:")
print("La calificación promedio por línea de producto y género se calculó agrupando por ambas columnas y sacando el promedio de la calificación.")
display(calif_promedio)

In [ ]:
print("5. ¿Cuántos productos de belleza se vendieron a hombres y mujeres en Yangon?")
belleza_yangon = df[(df['Ciudad'] == 'Yangon') & (df['Linea de producto'] == 'Health and beauty')]
ventas_belleza_genero = belleza_yangon.groupby('Género')['Cantidad'].sum()
print("Respuesta:")
print("La cantidad de productos de belleza vendidos a hombres y mujeres en Yangon se obtuvo "
      "filtrando por ciudad y línea de producto, y sumando las cantidades por género.")
display(ventas_belleza_genero)

In [ ]:
print("6. ¿Cuál fue el monto promedio de las compras de las mujeres Member?")
mujeres_member = df[(df['Género'] == 'Female') & (df['Tipo de comprador'] == 'Member')]
promedio_mujeres_member = mujeres_member['Total'].mean()
print("Respuesta:")
print(f"El monto promedio de las compras de las mujeres Member es: {promedio_mujeres_member:.2f}")
print("Esto se obtuvo filtrando por género femenino y tipo de cliente 'Member', luego calculando el promedio del valor total de la compra.")

In [ ]:
# Parte 10
import matplotlib.pyplot as plt

# 1. Ventas diarias promedio por línea de producto
plt.figure(figsize=(10, 6))
# Make sure 'promedio_ventas_diarias' is calculated in a previous cell
promedio_ventas_diarias.plot(kind='bar', color='skyblue')
plt.title('Ventas diarias promedio por línea de producto')
plt.xlabel('Línea de producto')
plt.ylabel('Ventas diarias promedio')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('ventas_diarias_promedio.png')
plt.show()

# 2. Calificación promedio por línea de producto y género
plt.figure(figsize=(12, 6))
# Make sure 'calif_promedio' is calculated in a previous cell
calif_promedio.plot(kind='bar', color=['lightcoral', 'lightblue']) # Use the correct variable name
plt.title('Calificación promedio por línea de producto y género')
plt.xlabel('Línea de producto')
plt.ylabel('Calificación promedio')
plt.xticks(rotation=45, ha='right')
plt.legend(calif_promedio.columns) # Use the columns of calif_promedio for the legend
plt.tight_layout()
plt.savefig('calificacion_promedio.png')
plt.show()

# 3. Ingresos totales por línea de producto
plt.figure(figsize=(10, 6))
# Make sure 'ingresos_por_linea' is calculated in a previous cell
ingresos_por_linea.plot(kind='bar', color='lightgreen')
plt.title('Ingresos totales por línea de producto')
plt.xlabel('Línea de producto')
plt.ylabel('Ingresos totales')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('ingresos_totales.png')
plt.show()

# 4. Productos de belleza vendidos en Yangon por género
plt.figure(figsize=(8, 6))
# Make sure 'ventas_belleza_genero' is calculated in a previous cell
ventas_belleza_genero.plot(kind='bar', color=['pink', 'lightblue'])
plt.title('Productos de belleza vendidos en Yangon por género')
plt.xlabel('Género')
plt.ylabel('Cantidad vendida')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('productos_belleza_yangon.png')
plt.show()